# 1. Getting Started with PyLapi

Python Lightweight API (PyLapi) is a Python API builder. It takes only a few seconds to automatically generate a Python API from OpenAPI specifications or less than an hour for an experienced Python programmer to create a custom Python API.

The auto-generated Python API contains fully functional resource classes with API methods embedded with OpenAPI specifications as inline comments, so you don't need to read the lengthy OpenAPI Specification to study or use the API.

---
You can also manually "rewrite" some of the generated classes and methods in a separate code file, for example, by specifying custom parameters, pre-processing the request, or post-processing the response. The PyLapi generator will merge your code into the generated Python API.

PyLapi's code-rewrite feature allows you to specify different API URLs for individual resources, effectively combining multiple API backend services into a single client API. All parameters and the request body are dynamically mapped based on the API method route, with the option to rewrite them if you so choose.

---
When OpenAPI Specifications are not available or you want to create a Python API your own way, you can inherit your API class from PyLapi, which does all the heavy lifting for you, so your code would mostly be a `pass`.

If you want to manipulate the API requests and responses, only minimal coding is required. In the tutorial [A ChatGPT Conversation with PyLapi](./3.%20A%20ChatGPT%20Conversation%20with%20PyLapi.ipynb), a fully functional Conversation resource is completed in 39 lines, including space lines for readability.

Here is what a PyLapi-supported Python API would look like:

```python
from pylapi import PyLapi

class MyAPI(PyLapi):
    def __init__(self, *args, **kwargs) -> None:
        super().__init__(*args, **kwargs)
        self.api_url = "https://api.example.com"
        self.api_auth_type = "Bearer"

# resource_name="invoice", resource_base_route"invoices"
@MyAPI.resource_class("invoice", "invoices")
class InvoiceResource(MyAPI):

    @MyAPI.resource_method("{invoice_number}", http_method="GET")
    def getInvoice(self): pass
    # To call: MyAPI.resource("invoice").getInvoice(...)
    # Request: GET https://api.example.com/invoices/{invoice_number}
```

The `MyAPI` API root class inherits the decorators `@MyAPI.resource_class` and `@MyAPI.resource_methods` from `PyLapi`, which prepares the API parameters and the request body in the API request, sends the request to the API URL route, processes the response, and handles the resource data for you.

---
The PyLapi framework makes it very easy and natural for you to use a PyLapi-supported Python API. For example,

```python
from myapi import MyAPI

MyAPI.auth(AUTH_ACCESS_TOKEN)

invoice = MyAPI.resource("invoice")
invoice.data = invoice.getInvoice("12345678") # using positional arguments
print(invoice)

item_list = invoice.getItemList()

product = MyAPI.resource("product")
invoice_total = 0.0
for item in item_list:
    product.data.number = item["product_number"]
    product.data = product.getProduct() # using implicit arguments from product.data
    subtotal = product.data.price * item["qty"]
    invoice_total += subtotal
    print(f"Item: {product.data.name}")
    print(f"  Price: ${product.data.price:0.2f}")
    print(f"  Qty: {item['qty']}")
    print(f"  Subtotal: ${subtotal:0.2f}")
print(f"Invoice Total: ${invoice_total:0.2f}")
```

## Install PyLapi

Please follow these instructions to install PyLapi and its generator, replacing all variables accordingly.


To install the PyLapi class:
```bash
pip install pylapi
```

To install the PyLapi generator:

```bash
cd $PATH_TO_DEV_ENV
git clone https://github.com/jackyko8/pylapi.git
cd pylapi
cp tools/pylapi_gen* $PATH_TO_BIN
```

To generate a PyLapi supported Python API:

```bash
cd $PATH_TO_MYAPI
cp $PATH_TO_DEV_ENV/pylapi/tools/pylapi_config_template.py myapi_config.py
# Configure myapi_config.py
pylapi_gen myapi_config.py
# Output
# MyAPI generated and saved in ./myapi.py
```

## Automatic API Generation

In this tutorial, we use the [GitHub REST API](https://docs.github.com/en/rest) as an example to illustrate how to automatically generate a Python API using PyLapi.

The OpenAPI Specification (OAS) of the GitHub v3 REST API can be found [online](https://github.com/github/rest-api-description/tree/main/descriptions/api.github.com).
For demonstration purposes, we have downloaded the OAS JSON document [here](./api.github.com.json).

### Configuration

Let us examine the content of [./gapi_config.py](./gapi_config.py).

In [1]:
!cat ./gapi_config.py

# gAPI (GitHub API) PyLapi Generator Configuration

api_class_name = "gAPI"
output_py_name = "./gapi.py"
oas_file_name = "./api.github.com.json"

guide_attrs = {
    "summary",
    "description",
    "parameters",
    "request_body",
    # "ref",
}

naming = {
    "class_name": "upperCamel($.path_segments[0]) + 'Resource'",
    "resource_name": "snake($.path_segments[0])",
    "class_path": "$.path_segments[0]",
    "resource_path": "'/'.join($.path_segments[1:])",
    "method_name": "lowerCamel($.operation_id)",
}

# code_rewrite_file_name = "./gapi_rewrite.py"


In the above configuration example, we are generating an API class called `gAPI` and storing the Python script in [./gapi.py](./gapi.py).
The OAS (OpenAPI Specification) is based on [./api.github.com.json](./api.github.com.json).

We also include some OAS attributes as a user guide embedded in the Python script as inline comments.
(You may uncomment "ref" to dereference the OAS attributes if you don't mind a lengthy script file.)

For `naming`, let us use the "Get repository content" method to illustrate. It is the method at line 31692 in the [OAS file](./api.github.com.json), or you may search for the API path `/repos/{owner}/{repo}/contents/{path}` or the operationId `repos/get-content`.

| Item          | Conversion                               | Value                          |
| ------------- | ---------------------------------------- | ------------------------------ |
| class_name    | upperCamel($.path_segments[0]) + 'Resource' | ReposResource                  |
| resource_name | snake($.path_segments[0])                   | repos                          |
| class_path    | $.path_segments[0]                          | repos                          |
| resource_path | '/'.join($.path_segments[1:])               | {owner}/{repo}/contents/{path} |
| method_name   | lowerCamel($.operation_id)               | reposGetContent                |

Notes:
1. The `$.` prefix indicates a method attribute, which we will discuss in later tutorials.
2. The naming specifications are processed by `eval`, so if you want to specify a blank `''`, use `"''"`. For example, `"resource_name": "''"`.

### Generation

In [2]:
!../tools/pylapi_gen gapi_config.py

gAPI generated and saved in ./gapi.py


Voilà! The `gAPI` Python API has been generated for you in seconds.
You may now open [./gapi.py](./gapi.py) and search for `RepoResource` and `reposGetContent` to verify if the naming is right.

---
In the next tutorial, we are going to show you [how to use a PyLapi API](2.%20How%20to%20use%20a%20PyLapi%20API.ipynb) using this newly generated `gAPI` as an example.

## End of page